In [1]:
# https://github.com/mojyt/cpptraj/blob/master/test/Test_Hbond/RunTest.sh
# hbond detail: http://ambermd.org/doc12/Amber15.pdf (page 572)
import pytraj.io as io

# system: tripzip2 peptide, explicit water with `ortho` box
traj = io.load("../tests/data/tz2.ortho.nc", "../tests/data/tz2.ortho.parm7")
print (traj)
print (traj[0].box)

<TrajReadOnly with 10 frames, 5293 atoms/frame>
           
<Box: ortho, (x, y, z, alpha, beta, gamma) = (35.2627796623, 41.8455476799, 36.168629529, 90.0, 90.0, 90.0)>


In [2]:
command = r"out solvhb.dat avgout solvavg.dat :1-13 solventacceptor :WAT@O solventdonor :WAT \
            solvout solvavg.dat bridgeout solvavg.dat"

# do hbond search
dslist = traj.search_hbonds(command)
print (dslist)

In [3]:
print (dslist.get_legends())

[u'HB_00000[UU]', u'HB_00000[UV]', u'HB_00000[Bridge]', u'HB_00000[ID]', u'LYS_12@O-SER_1@N-H3', u'LYS_8@O-GLU_5@N-H', u'GLU_5@O-LYS_8@N-H', u'SER_1@O-LYS_12@N-H', u'GLU_5@OE2-LYS_12@NZ-HZ1', u'SER_1@H2-V', u'SER_1@HG-V', u'TRP_2@H-V', u'THR_3@HG1-V', u'ASN_6@H-V', u'ASN_6@HD22-V', u'LYS_8@HZ1-V', u'LYS_8@HZ2-V', u'TRP_9@HE1-V', u'THR_10@HG1-V', u'LYS_12@HZ2-V', u'TRP_11@O-V', u'THR_10@OG1-V', u'GLU_5@OE2-V', u'GLU_5@O-V', u'THR_3@OG1-V', u'TRP_2@O-V', u'GLU_5@OE1-V', u'TRP_9@O-V', u'GLY_7@O-V', u'ASN_6@O-V', u'ASN_6@OD1-V', u'TRP_4@O-V', u'GLU_5@OE1-LYS_12@NZ-HZ1', u'LYS_12@HZ3-V', u'NHE_13@HN1-V', u'THR_10@O-THR_3@N-H', u'TRP_2@HE1-V', u'TRP_4@H-V', u'ASN_6@HD21-V', u'GLY_7@H-V', u'LYS_8@HZ3-V', u'TRP_9@H-V', u'SER_1@OG-V', u'TRP_2@O-THR_3@OG1-HG1', u'SER_1@H1-V', u'TRP_4@HE1-V', u'TRP_11@HE1-V', u'THR_3@O-THR_10@N-H', u'TRP_11@H-V']


In [4]:
dslist.groupby("SER_1").to_dict()

{u'LYS_12@O-SER_1@N-H3': [1, 1, 1, 1, 0, 1, 0, 1, 0, 1],
 u'SER_1@H1-V': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 u'SER_1@H2-V': [1, 1, 1, 0, 0, 1, 1, 0, 1, 0],
 u'SER_1@HG-V': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 u'SER_1@O-LYS_12@N-H': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 u'SER_1@OG-V': [0, 0, 0, 1, 0, 1, 1, 0, 1, 1]}

In [5]:
from pytraj.misc import to_amber_mask

mask = to_amber_mask('LYS_12@O-SER_1@N-H3')
print (mask)

d0_dis = traj.calc_distance(mask)
d1_angle = traj.calc_angle(mask + " @H3")
print ('hbond distance: ',d0_dis.tolist())

print ("")
print ('hbond angle: ', d1_angle.tolist())

print ("")
print ("agv, min, max")
print (d0_dis.avg(), d0_dis.min(), d0_dis.max())

:12@O :1@N
('hbond distance: ', [2.785477022697817, 2.8315799255658454, 2.7679902368377243, 2.867518628010597, 3.101924965773696, 2.8775658495960945, 2.7061607917287405, 2.9813771202941988, 3.14691744364174, 2.8054902050147694])

('hbond angle: ', [12.366420503543578, 12.026350957751992, 23.086008841650138, 28.254980253025952, 31.58306803649342, 18.318812722365806, 30.68882165123469, 24.330592665923163, 20.627688022414265, 5.128303656447668])

agv, min, max
(2.8872002189161226, 2.7061607917287405, 3.14691744364174)


In [6]:
print (dslist.get_aspects())

dstring = dslist.groupby('ID', mode='aspect')[0]
from pytraj.misc import get_atts

print (dstring.to_ndarray())
# https://github.com/mojyt/cpptraj/blob/master/test/Test_Hbond/solvhb.dat.save

set([u'Bridge', u'UU', u'UV', u'solventhb', u'solutehb', u'ID'])
[u'208(10+11+),494(2+3+),1409(3+5+),' u'208(10+11+),494(2+3+),1409(3+5+),'
 u'208(10+11+),496(5+12+),' u'208(10+11+),496(5+12+),' u'496(5+12+),'
 u'496(5+12+),' u'None' u'None' u'1384(5+10+),' u'266(10+11+),1384(5+10+),']


In [7]:
print ("hb[UU]", dslist.groupby("UU").tolist())
print ("hb[UV]", dslist.groupby("UV").tolist())
print ("hb[Bridge]", dslist.groupby("Bridge").tolist())
# https://github.com/mojyt/cpptraj/blob/master/test/Test_Hbond/solvhb.dat.save

('hb[UU]', [[5, 5, 5, 4, 4, 5, 6, 5, 5, 4]])
('hb[UV]', [[27, 27, 31, 26, 30, 28, 21, 24, 24, 28]])
('hb[Bridge]', [[3, 3, 2, 2, 1, 1, 0, 0, 1, 2]])


#### Raw output from cpptraj
```
#Frame         hb[UU]       hb[UV]   hb[Bridge]                            hb[ID]
       1            5           27            3 208(10+11+),494(2+3+),1409(3+5+),
       2            5           27            3 208(10+11+),494(2+3+),1409(3+5+),
       3            5           31            2           208(10+11+),496(5+12+),
       4            4           26            2           208(10+11+),496(5+12+),
       5            4           30            1                       496(5+12+),
       6            5           28            1                       496(5+12+),
       7            6           21            0                              None
       8            5           24            0                              None
       9            5           24            1                      1384(5+10+),
      10            4           28            2          266(10+11+),1384(5+10+),
```

In [8]:
dslist.groupby("SER").to_dataframe()

,LYS_12@O-SER_1@N-H3,SER_1@H1-V,SER_1@H2-V,SER_1@HG-V,SER_1@O-LYS_12@N-H,SER_1@OG-V
0,1,0,1,1,1,0
1,1,0,1,1,1,0
2,1,0,1,1,1,0
3,1,0,0,1,1,1
4,0,1,0,1,1,0
5,1,0,1,1,1,1
6,0,0,1,1,1,1
7,1,0,0,1,1,0
8,0,0,1,1,1,1
9,1,0,0,1,1,1
